## 이전 코드 세팅

In [3]:
import feedparser
from urllib.parse import quote
from typing import List, Dict, Optional

class GoogleNews:
    """
    구글 뉴스를 검색하고 결과를 반환하는 클래스입니다.
    """

    def __init__(self):
        self.base_url = "https://news.google.com/rss"

    def _fetch_news(self, url: str, k: int = 3) -> List[Dict[str, str]]:
        news_data = feedparser.parse(url)
        return [
            {"title": entry.title, "link": entry.link}
            for entry in news_data.entries[:k]
        ]

    def _collect_news(self, news_list: List[Dict[str, str]]) -> List[Dict[str, str]]:

        if not news_list:
            print("해당 키워드의 뉴스가 없습니다.")
            return []

        result = []
        for news in news_list:
            result.append({"url": news["link"], "content": news["title"]})

        return result

    def search_latest(self, k: int = 3) -> List[Dict[str, str]]:
        url = f"{self.base_url}?hl=ko&gl=KR&ceid=KR:ko"
        news_list = self._fetch_news(url, k)
        return self._collect_news(news_list)

    def search_by_keyword(
        self, keyword: Optional[str] = None, k: int = 3
    ) -> List[Dict[str, str]]:
        if keyword:
            encoded_keyword = quote(keyword)
            url = f"{self.base_url}/search?q={encoded_keyword}&hl=ko&gl=KR&ceid=KR:ko"
        else:
            url = f"{self.base_url}?hl=ko&gl=KR&ceid=KR:ko"
        news_list = self._fetch_news(url, k)
        return self._collect_news(news_list)

In [4]:
from dotenv import load_dotenv
from typing import Annotated, List, Dict
from typing_extensions import TypedDict

from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

load_dotenv()

class State(TypedDict):
    messages: Annotated[list, add_messages]

news_tool = GoogleNews()


@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)


tools = [search_keyword]

llm = ChatOpenAI(model="gpt-4o-mini")

llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)

graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

graph_builder.add_edge("tools", "chatbot")

graph_builder.add_edge(START, "chatbot")

graph_builder.add_edge("chatbot", END)

memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory)

## tools가 호출이 일어나면 멈추는 stream 구성 (이전과 동일)

In [5]:
from langchain_teddynote.messages import pretty_print_messages
from langchain_core.runnables import RunnableConfig

question = "AI 관련 최신 뉴스를 알려주세요."

input = State(messages=[("user", question)])

config = RunnableConfig(
    recursion_limit=10,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "1"}, 
    tags=["my-rag"],  
)

for event in graph.stream(
    input=input,
    config=config,
    stream_mode="values",
    interrupt_before=["tools"],  #interrupt before을 선택했기 때문에 tools가 호출이 일어나는 순간 멈춤
):
    for key, value in event.items():

        print(f"\n[{key}]\n")

        pretty_print_messages(value)

        if "messages" in value:
            print(f"메시지 개수: {len(value['messages'])}")


[messages]

================================ Human Message =================================

AI 관련 최신 뉴스를 알려주세요.

[messages]

================================ Human Message =================================

AI 관련 최신 뉴스를 알려주세요.
================================== Ai Message ==================================
Tool Calls:
  search_keyword (call_Ehgr6C0f3kdqhRg4sFaQ5kum)
 Call ID: call_Ehgr6C0f3kdqhRg4sFaQ5kum
  Args:
    query: AI


### 다음 노드가 tools인 것을 볼 수 있음

In [7]:
snapshot = graph.get_state(config)

# 다음 스냅샷 상태
snapshot.next

('tools',)

### 만약 아무 조건을 걸고 싶지 않다면 None을 넣어주면 됨

In [8]:
# `None`는 현재 상태에 아무것도 추가하지 않음
events = graph.stream(None, config, stream_mode="values")

# 이벤트 반복 처리
for event in events:
    # 메시지가 이벤트에 포함된 경우
    if "messages" in event:
        # 마지막 메시지의 예쁜 출력
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  search_keyword (call_Ehgr6C0f3kdqhRg4sFaQ5kum)
 Call ID: call_Ehgr6C0f3kdqhRg4sFaQ5kum
  Args:
    query: AI
================================= Tool Message =================================
Name: search_keyword

[{"url": "https://news.google.com/rss/articles/CBMiYEFVX3lxTE93ZzFNeGNPb1dsbU9iUmwtbi1EcGZnekRZWkNTd1RiaDRMaXk2dnRKTWhJbWh2NGxwc1gxSG96dm8xR1BOS2hHbmlnc0EtZlJkTW93bTNudmxEazE3YVg2dNIBeEFVX3lxTFB3d1JfVl9XVFRUWEVVR1NZbURtVEdRZ1JvRHdZbkJVQzNhMUF6Mm5pckZMeTFQUlR0X09zY201YWI1eTJSZjdPMU1UYlJvZnBPMHNEVkpkSDFzMmhBWDBaem5xT08xRjc4NEV3Z3pScG1sXzIyYkU5VQ?oc=5", "content": "대덕전자, AMD향 'AI 가속기' MLB 공급 최종승인 앞둬 - 뉴시스"}, {"url": "https://news.google.com/rss/articles/CBMiYEFVX3lxTE9DRnpMTlFBQ2hQQkctalB2LWQ1aFhhNVFFNzZnbXVPcnE3MndSVWRxRTk2S2FhWkFja21QeDVYTUhOVGNnVXl4YkY1eXgtdXZRWlpWckJYOVVYUUFqTlhUMQ?oc=5", "content": "‘AI 지우기’ 아이폰·갤럭시 대결…‘인생샷’에 찍힌 관광객 지워봤다 - 한겨레"}, {"url": "https://news.google.com/rss

to_replay로 상태 저장

In [10]:
to_replay = None

# 상태 기록 가져오기
for state in graph.get_state_history(config):
    # 메시지 수 및 다음 상태 출력
    print(state.values["messages"])
    print("메시지 수: ", len(state.values["messages"]), "다음 노드: ", state.next)
    print("-" * 80)
    # 특정 상태 선택 기준: 채팅 메시지 수
    if len(state.values["messages"]) == 3:

        to_replay = state

[HumanMessage(content='AI 관련 최신 뉴스를 알려주세요.', additional_kwargs={}, response_metadata={}, id='f8f72246-7671-4e5a-972c-9378ce6108c4'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Ehgr6C0f3kdqhRg4sFaQ5kum', 'function': {'arguments': '{"query":"AI"}', 'name': 'search_keyword'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 51, 'total_tokens': 65, 'completion_tokens_details': {'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-90cdb57e-d621-4f27-9ec0-7be12ead1b8b-0', tool_calls=[{'name': 'search_keyword', 'args': {'query': 'AI'}, 'id': 'call_Ehgr6C0f3kdqhRg4sFaQ5kum', 'type': 'tool_call'}], usage_metadata={'input_tokens': 51, 'output_tokens': 14, 'total_tokens': 65}), ToolMess

to_replay 저장된 상태를 stream에 넣으면 그 상태부터 이어서 진행할 수 있음

In [11]:
for event in graph.stream(None, to_replay.config, stream_mode="values"):
    # 메시지가 이벤트에 포함된 경우
    if "messages" in event:
        # 마지막 메시지 출력
        event["messages"][-1].pretty_print()

================================= Tool Message =================================
Name: search_keyword

[{"url": "https://news.google.com/rss/articles/CBMiYEFVX3lxTE93ZzFNeGNPb1dsbU9iUmwtbi1EcGZnekRZWkNTd1RiaDRMaXk2dnRKTWhJbWh2NGxwc1gxSG96dm8xR1BOS2hHbmlnc0EtZlJkTW93bTNudmxEazE3YVg2dNIBeEFVX3lxTFB3d1JfVl9XVFRUWEVVR1NZbURtVEdRZ1JvRHdZbkJVQzNhMUF6Mm5pckZMeTFQUlR0X09zY201YWI1eTJSZjdPMU1UYlJvZnBPMHNEVkpkSDFzMmhBWDBaem5xT08xRjc4NEV3Z3pScG1sXzIyYkU5VQ?oc=5", "content": "대덕전자, AMD향 'AI 가속기' MLB 공급 최종승인 앞둬 - 뉴시스"}, {"url": "https://news.google.com/rss/articles/CBMiYEFVX3lxTE9DRnpMTlFBQ2hQQkctalB2LWQ1aFhhNVFFNzZnbXVPcnE3MndSVWRxRTk2S2FhWkFja21QeDVYTUhOVGNnVXl4YkY1eXgtdXZRWlpWckJYOVVYUUFqTlhUMQ?oc=5", "content": "‘AI 지우기’ 아이폰·갤럭시 대결…‘인생샷’에 찍힌 관광객 지워봤다 - 한겨레"}, {"url": "https://news.google.com/rss/articles/CBMiakFVX3lxTE1aRm1RR2FoWW9Ld3EyTkd2RlF2NlhfV0RwS0l6VW9BTlBDYUpQVnZ3ay1rRDZwTWVYTHZNR3AwSFI4dmhVVnlielJwcXFjU1RPNDM3YmkwaVJ3WU5rd0dPYjhMZC1sLUI4UUE?oc=5", "content": "[11월5일] 텍스트보다 이미지가 편향 심각...